In [1]:
import tensorflow as tf
import os
%config Completer.use_jedi=False

In [2]:
!ls ../cats_n_dogs/train

cat  dog


In [3]:
petimages='../cats_n_dogs/train'

In [4]:
dataset=tf.data.Dataset.list_files(petimages+'*/*/*')
for f in dataset.take(5):
    print(f)

tf.Tensor(b'../cats_n_dogs/train/dog/7106.jpg', shape=(), dtype=string)
tf.Tensor(b'../cats_n_dogs/train/dog/2350.jpg', shape=(), dtype=string)
tf.Tensor(b'../cats_n_dogs/train/cat/8227.jpg', shape=(), dtype=string)
tf.Tensor(b'../cats_n_dogs/train/dog/12381.jpg', shape=(), dtype=string)
tf.Tensor(b'../cats_n_dogs/train/dog/4209.jpg', shape=(), dtype=string)


In [5]:
def parse_imgpath(filename):
    label=tf.strings.split(filename,os.sep)[-2]
    image = tf.io.read_file(filename)
#     image = tf.image.decode_jpeg(image)
    image=tf.image.decode_image(image,expand_animations=False,channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [124, 124])
    if label=='cat':
        label=tf.constant(1)
    else:
        label=tf.constant(0)
#     label=parse_label(label)
    return image, label

In [6]:
dataset=dataset.map(parse_imgpath)

In [7]:
for f in dataset.take(5):
    print(f[0].shape,f[1].numpy())
#     print(f)

(24, 24, 3) 0
(24, 24, 3) 0
(24, 24, 3) 0
(24, 24, 3) 0
(24, 24, 3) 1


In [8]:
def one_hot(image, label):
    """
    Converts the label to categorical.
    Arguments ~
        image: Tensor of Shape (224,224,3) - Simply for outputting
        label: Tensor of Shape (32,) for casting and converting to categorical
    Returns the image (as it was inputted) and the label converted to a categorical vector
    """
    # Casts to an Int and performs one-hot ops
    label = tf.one_hot(tf.cast(label, tf.int32), 2)
    # Recasts it to Float32
#     label = tf.cast(label, tf.float32)
    return image, label

In [9]:
dataset=dataset.map(one_hot)

In [10]:
for f in dataset.take(1):
    print(f[0].shape,f[1])

(24, 24, 3) tf.Tensor([0. 1.], shape=(2,), dtype=float32)


In [11]:
# def scale_image(image,label):
#     image=tf.divide(image[0],tf.constant(1,dtype=tf.float32)) # images are already scaled,if not change 1 to 255
#     return image,label

In [12]:
# dataset=dataset.map(scale_image)
# for f in dataset.take(1):
#     print(f)

In [13]:
dataset=dataset.shuffle(buffer_size=100).batch(32)


In [14]:
for f in dataset.take(1):
    print(f[0].shape,f[1])

(32, 24, 24, 3) tf.Tensor(
[[0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]], shape=(32, 2), dtype=float32)


In [15]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=(
    3, 3), activation='relu', input_shape=(24, 24, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(
    filters=64, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(
    filters=128, kernel_size=(3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(2, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 22, 22, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 9, 9, 64)          9280      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 2, 2, 128)         73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 1, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 128)               0

In [16]:
# model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.compile(optimizer='adam',
                  loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
r = model.fit(
dataset,batch_size=32
)

InvalidArgumentError:  Input size should match (header_size + row_size * abs_height) but they differ by 2
	 [[{{node decode_image/DecodeImage}}]]
	 [[IteratorGetNext]] [Op:__inference_train_function_986]

Function call stack:
train_function


In [ ]:
dataset.